In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate,ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1 
)

template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b},", 
)

prompt = template.format(country_a="Mexico",country_b="Thailand")

chat.predict(prompt)


'The distance between Mexico and Thailand is approximately 9,500 miles (15,300 kilometers) when measured in a straight line.'

In [7]:
from langchain.schema import HumanMessage,AIMessage,SystemMessage


messages = [
    SystemMessage(content="You are a geographic expert. and you only reply in {language}.") ,
    AIMessage(content="Ciao, mi chiamo {name}"),
    HumanMessage(content="What is the distance between {country_a} and {country_b}. Also, what is your name?"),
]

chat.predict_messages(messages)


AIMessage(content="Mi dispiace, ma non posso aiutarti con la distanza tra Messico e Thailandia. Sono un esperto di geografia dell'Italia. Il mio nome è Paolo. Posso aiutarti con informazioni sulla geografia italiana. Fammi sapere se hai bisogno di aiuto con qualcos'altro!")

In [15]:
template = ChatPromptTemplate.from_messages(
    [
    ("system","You are a geographic expert. and you only reply in {language}."),
    ("ai","Ciao, mi chiamo {name}"),
    ("human","What is the distance between {country_a} and {country_b}. Also, what is your name?")
    ]
)

prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a = "Mexico",
    country_b = "Thailand",
)

chat.predict_messages(prompt)

AIMessage(content='Γεια σας! Το όνομά μου είναι Σωκράτης. Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα.')

In [17]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    
    def parse(self,text):
        items = text.strip().split(",")
        return list(map(str.strip,items))

p = CommaOutputParser()
p.parse("Hello,how, are, you")


['Hello', 'how', 'are', 'you']

In [24]:
template = ChatPromptTemplate.from_messages(
    [
       ( "system"," You are a comma separated listing generating machine. Everything you are asked will be answered with a list of max {max_items} in lowercase. Do not reply with anlything else" ) ,
       ( "human","{question}" ) ,
    ]
)

prompt = template.format_messages(
    max_items=10,
    question="What are the planets?"
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()
p.parse(result.content)


AIMessage(content='Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, Neptune, Pluto')

In [28]:


chain = template | chat | CommaOutputParser()

chain.invoke({
     "max_items":5,
    "question":"What are the Pokemons?"
})



KeyboardInterrupt: 

In [39]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

chef_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a world-class international chef. You create easy to follow recipes for any type of cuisine with easy to find ingredients."),
    ("human","I want to cook {cuisine} food."),
])

chef_chain = chef_prompt | chat 




In [40]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human","{recipe}"),
])

veg_chain = veg_chef_prompt | chat 

final_chain = {"recipe":chef_chain} | veg_chain 

final_chain.invoke({
    "cuisine":"Indian",
})

Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Let's start with a classic and popular dish - Chicken Tikka Masala. Here's a simple recipe for you to try at home:

Chicken Tikka Masala

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tablespoons lemon juice
- 2 teaspoons ground cumin
- 2 teaspoons ground coriander
- 1 teaspoon ground turmeric
- 1 teaspoon chili powder
- 1 teaspoon paprika
- 1 teaspoon garam masala
- 2 cloves garlic, minced
- 1-inch piece of ginger, grated
- Salt and pepper to taste
- 2 tablespoons vegetable oil
- 1 onion, finely chopped
- 1 can (14 oz) tomato sauce
- 1 cup heavy cream
- Fresh cilantro, chopped (for garnish)

Instructions:
1. In a bowl, combine yogurt, lemon juice, cumin, coriander, turmeric, chili powder, paprika, garam masala, garlic, ginger, salt, and pepper. Add the chicken pieces and mix well to coat. Cover and marinate in the refrigerator for at least 1 hou

AIMessageChunk(content="For a vegetarian version of Chicken Tikka Masala, you can replace the chicken with a plant-based alternative such as tofu or paneer. Here's how you can prepare these alternatives:\n\n1. Tofu: Use firm or extra-firm tofu for this recipe. Drain the tofu and pat it dry with paper towels to remove excess moisture. Cut the tofu into bite-sized cubes and follow the same marinating process as you would with the chicken. Tofu absorbs flavors well, so marinating it for at least 1 hour will help enhance the taste. When baking the tofu, you can follow the same instructions as you would for the chicken.\n\n2. Paneer: Paneer is a type of Indian cottage cheese that holds its shape well when cooked. You can either make your own paneer at home or purchase it from the store. Cut the paneer into bite-sized cubes and add them to the marinade mixture. Since paneer is delicate, marinate it for a shorter time, around 30 minutes to 1 hour. When adding the paneer to the sauce, do so ge